<a href="https://colab.research.google.com/github/rushikesh6548/IPL_STATS/blob/main/Cricket_XI_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [37]:
df = pd.read_excel("/content/dataset_for_ML.xlsx")

In [7]:
df.describe()

,match_id,avg_runs_last_5_innings,avg_runs_last_5_innings_at_that_venue,avg_wickets_last_5,avg_wickets_last_5_innings_at_that_venue,fantasy_points
count,1.526500e+04,13301.000000,12596.000000,8149.000000,12596.000000,15265.000000
mean,1.157863e+06,17.703824,18.176606,1.577220,18.176606,43.058434
std,2.084185e+05,13.410841,15.690640,0.539308,15.690640,40.996638
min,7.292790e+05,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.082592e+06,6.500000,5.500000,1.000000,5.500000,12.000000
50%,1.216494e+06,15.800000,15.000000,1.500000,15.000000,30.000000
75%,1.304107e+06,26.400000,26.800000,2.000000,26.800000,60.000000
max,1.426312e+06,83.000000,113.000000,6.000000,113.000000,284.000000


In [8]:
df.dtypes

,0
match_id,int64
season,object
venue,object
player_name,object
player_team,object
opponent_team,object
avg_runs_last_5_innings,float64
avg_runs_last_5_innings_at_that_venue,float64
avg_wickets_last_5,float64
avg_wickets_last_5_innings_at_that_venue,float64


In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [38]:

# Fill missing values for numerical columns
df['avg_runs_last_5_innings'].fillna(0, inplace=True)
df['avg_wickets_last_5'].fillna(0, inplace=True)
df['avg_runs_last_5_innings_at_that_venue'].fillna(0, inplace=True)
df['avg_wickets_last_5_innings_at_that_venue'].fillna(0, inplace=True)

<ipython-input-38-c45d2faee3c9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['avg_runs_last_5_innings'].fillna(0, inplace=True)
<ipython-input-38-c45d2faee3c9>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [39]:
# Fill missing values for categorical columns
df.fillna("Unknown", inplace=True)


In [40]:
df.drop(columns=['match_id', 'season'], inplace=True)

In [41]:
# Define features and target
X = df.drop(columns=['fantasy_points'])  # Keep player_name and other categorical columns
y = df['fantasy_points']


In [42]:
nominal_categorical_cols = ['venue', 'player_team', 'opponent_team', 'player_name', 'pitch_nature']
numeric_cols = ['avg_runs_last_5_innings', 'avg_runs_last_5_innings_at_that_venue',
                'avg_wickets_last_5', 'avg_wickets_last_5_innings_at_that_venue']


In [43]:
# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), nominal_categorical_cols)
])


In [44]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

In [46]:
# Train and evaluate models
results = {}
trained_pipelines = {}
for name, model in models.items():
    pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {'RMSE': rmse, 'R² Score': r2}
    trained_pipelines[name] = pipe  # Store trained pipeline for inference

In [47]:
for model, metrics in results.items():
    print(f"{model}: RMSE = {metrics['RMSE']:.2f}, R² Score = {metrics['R² Score']:.2f}")


Linear Regression: RMSE = 34.01, R² Score = 0.29
Random Forest: RMSE = 36.44, R² Score = 0.18
XGBoost: RMSE = 34.86, R² Score = 0.25


In [49]:
def predict_fantasy_points(input_data, model_name='Random Forest'):
    if model_name not in trained_pipelines:
        raise ValueError("Model not found! Choose from: " + ", ".join(trained_pipelines.keys()))

    model = trained_pipelines[model_name]
    df_input = pd.DataFrame([input_data])  # Convert input to DataFrame
    df_input.fillna("Unknown", inplace=True)  # Handle missing categorical values
    df_input[numeric_cols] = df_input[numeric_cols].fillna(0)  # Handle missing numerical values
    prediction = model.predict(df_input)[0]  # Get prediction
    return prediction


In [63]:
# Example input for prediction
sample_input = {
    'venue': 'Wankhede Stadium',
    'player_team': 'Rajasthan Royals',
    'opponent_team': 'Mumbai Indians',
    'player_name': 'Jos Buttler',
    'pitch_nature': 'Balanced',
    'avg_runs_last_5_innings': 2,
    'avg_runs_last_5_innings_at_that_venue': 34,
    'avg_wickets_last_5': 0.0,
    'avg_wickets_last_5_innings_at_that_venue': 0.0
}

# Predict fantasy points for the sample input
predicted_points = predict_fantasy_points(sample_input, model_name='Random Forest')
print(f"Predicted Fantasy Points: {predicted_points:.2f}")




Predicted Fantasy Points: 49.21
